# A* Pathfinding v3: Grid Environment (Power Cost)

This notebook demonstrates the use of the `GridEnvironment` with a `PowerCost` function. This is the most advanced version, suitable for rover navigation where slope steepness is a critical factor. The cost function penalizes steep slopes exponentially.

## Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add project root to path
project_root = Path("../").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.terrain_navigate import AStar, GridEnvironment, PowerCost

## 1. Generate Synthetic Terrain Grid
We'll create a 2D heightmap (Z) representing the terrain.

In [ ]:
# Create a 100x100 grid
x = np.linspace(0, 100, 100)
y = np.linspace(0, 100, 100)
X, Y = np.meshgrid(x, y)

# Generate terrain: hills + a large obstacle
Z = np.sin(X/10) * np.cos(Y/10) * 5 + 10
Z[40:60, 40:60] += 20.0 # Obstacle

print(f"Grid shape: {Z.shape}")

plt.figure(figsize=(8, 6))
plt.imshow(Z, cmap='terrain', origin='lower', extent=[0, 100, 0, 100])
plt.colorbar(label='Elevation (m)')
plt.title("Synthetic Terrain Grid")
plt.show()

## 2. Initialize Grid Environment and Power Cost
The `PowerCost` function takes parameters `n` (exponent) and `alpha` (scaling factor) to tune the penalty for slopes.

In [ ]:
env = GridEnvironment(
    Z=Z,
    global_x_min=0.0,
    global_x_max=100.0,
    global_y_min=0.0,
    global_y_max=100.0,
    resolution=1.0
)

# Power Cost: Cost = Distance * (1 + alpha * slope^n)
cost_fn = PowerCost(n=2.0, alpha=10.0)
astar = AStar()

## 3. Plan Path

In [ ]:
start_raw = np.array([10.0, 10.0, 0.0])
goal_raw = np.array([90.0, 90.0, 0.0])

# Snap to nearest grid nodes
start_node = env.get_nearest_node(start_raw)
goal_node = env.get_nearest_node(goal_raw)

print(f"Start Node: {start_node}")
print(f"Goal Node: {goal_node}")

# Find path
path, cost = astar.find_path(start_node, goal_node, env, cost_fn, node_radius=1.5)

if path is None:
    print("No path found!")
else:
    print(f"Path found! Length: {len(path)} nodes, Cost: {cost:.2f}")

## 4. Visualize Path

In [ ]:
if path is not None:
    path_arr = np.array(path)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(Z, cmap='terrain', origin='lower', extent=[0, 100, 0, 100])
    plt.colorbar(label='Elevation (m)')
    
    plt.plot(path_arr[:, 0], path_arr[:, 1], 'r-', linewidth=2, label='Path')
    plt.plot(start_node[0], start_node[1], 'go', markersize=10, label='Start')
    plt.plot(goal_node[0], goal_node[1], 'rx', markersize=10, label='Goal')
    
    plt.legend()
    plt.title("A* Path on Grid (Power Cost)")
    plt.xlabel("X (m)")
    plt.ylabel("Y (m)")
    plt.show()